In [2]:
import re
import sys
import os.path
import pickle
import pandas as pd
import numpy as np
# import matplotlib.pyplot as plt
import fastf1
import time
import json

# Spark imports
from pyspark.rdd import RDD
from pyspark.sql import DataFrame
from pyspark.sql import SparkSession
from pyspark.sql.functions import desc
from pyspark.sql.types import *

from helper import pitstopLap, buildDataFrame, getResults

<h1> Dependency Check and Set Up

In [3]:
modulename = ['pandas','numpy','matplotlib','fastf1']
for module in modulename:
    if module not in sys.modules:
        print('You have not imported the {} module'.format(module))

def init_spark():
    spark = SparkSession \
        .builder \
        .appName("Python Spark SQL basic example") \
        .config("spark.some.config.option", "some-value") \
        .getOrCreate()
    return spark

spark = init_spark()

raceResultPoints = {1:25,2:18,3:15,4:12,5:10,6:8,7:6,8:4,9:2, 10:1,11:0,12:0,13:0,14:0,15:0,16:0,17:0,18:0,19:0, 20:0}

driverCost = {}
with open(f"Post Week 1 Cost.json") as f:
    driverCostJSON = json.load(f)
for i in range(len(driverCostJSON)):
    driverCost[driverCostJSON[i]["Driver"]] = driverCostJSON[i]["Cost"]


You have not imported the matplotlib module


In [ ]:
def racePoints(dataframe, driver):
    # endPoints = raceResultPoints.get(dataframe.Driver.loc[driver])
    points = []
    for i in range(len(dataframe)):
        if not(re.search(r"(Finished)|(Lap)", dataframe.Status[i])):
            points.append(-20)
        # elif "Lap" in dataframe.Status[i]:
        #     points.append(-20)
        else:
            points.append(raceResultPoints.get(int(i+1))+(dataframe.Start[i]-dataframe.End[i]))

    dataframe["Points"] = points
    return dataframe

def pointsPerMillion(dataframe,driver):
    df = racePoints(dataframe,driver)
    pointsPerMillion = []
    for i in range(len(dataframe)):
        try:
            pointsPerMillion.append(df.Points[i]/driverCost.get(df.Driver[i]))
        except:
            pointsPerMillion.append(-1000)

    dataframe["PointsPerMillion"] = pointsPerMillion
    return dataframe


<h1> Goals

1) Refine the Points Per Million metric better then
2) Use Spark dataframes to process years of data to get Points Per Million
3) Use Race as x-axis, PointsPerMillion as y-axis, Driver as label


In [9]:
time1 = time.time()
f1Races = [    'Australian Grand Prix',    'Bahrain Grand Prix',    'Vietnamese Grand Prix',    'Chinese Grand Prix',    'Dutch Grand Prix',    'Spanish Grand Prix',    'Monaco Grand Prix',    'Azerbaijan Grand Prix',    'Canadian Grand Prix',    'French Grand Prix',    'Austrian Grand Prix',    'British Grand Prix',    'Hungarian Grand Prix',    'Belgian Grand Prix',    'Dutch Grand Prix',    'Italian Grand Prix',    'Russian Grand Prix',    'Singapore Grand Prix',    'Japanese Grand Prix',    'United States Grand Prix',    'Mexican Grand Prix',    'Brazilian Grand Prix',    'Qatar Grand Prix',    'Saudi Arabian Grand Prix',    'Abu Dhabi Grand Prix']

allRaces = []
for race in f1Races:
    try:
        allRaces.append(getResults((2022, race , 'R')))
    except Exception as e:
        print(e)
# allRaces=getResults((2022, 'Australian Grand Prix' , 'R'))
allRaces = pd.concat(allRaces)
allRaces.to_parquet("result.parquet.gzip", compression="gzip")

# pointsDF = racePoints(allRaces,"LEC")
# value = pointsPerMillion(allRaces,"ZHO")
# value = pointsPerMillion(getResults((2022, 'Saudi Arabian Grand Prix' , 'R')),"ZHO")
print(f"{time.time()-time1} seconds")

core           INFO 	Loading data for Australian Grand Prix - Race [v2.3.0]
api            INFO 	Using cached data for driver_info
api            INFO 	Using cached data for timing_data
api            INFO 	Using cached data for timing_app_data
core           INFO 	Processing timing data...
api            INFO 	Using cached data for session_status_data
api            INFO 	Using cached data for track_status_data
api            INFO 	Using cached data for car_data
api            INFO 	Using cached data for position_data
api            INFO 	Using cached data for weather_data
api            INFO 	Using cached data for race_control_messages
core           INFO 	Finished loading data for 20 drivers: ['16', '11', '63', '44', '4', '3', '31', '77', '10', '23', '24', '18', '47', '20', '22', '6', '14', '1', '5', '55']
core           INFO 	Loading data for Bahrain Grand Prix - Race [v2.3.0]
api            INFO 	Using cached data for driver_info


unused columns exists that are not in the dataframe


api            INFO 	Using cached data for timing_data
api            INFO 	Using cached data for timing_app_data
core           INFO 	Processing timing data...
api            INFO 	Using cached data for session_status_data
api            INFO 	Using cached data for track_status_data
api            INFO 	Using cached data for car_data
api            INFO 	Using cached data for position_data
api            INFO 	Using cached data for weather_data
api            INFO 	Using cached data for race_control_messages
core           INFO 	Finished loading data for 20 drivers: ['16', '55', '44', '63', '20', '77', '31', '22', '14', '24', '47', '18', '23', '3', '4', '6', '27', '11', '1', '10']
core           INFO 	Loading data for São Paulo Grand Prix - Race [v2.3.0]
api            INFO 	Using cached data for driver_info


unused columns exists that are not in the dataframe


api            INFO 	Using cached data for timing_data
api            INFO 	Using cached data for timing_app_data
core           INFO 	Processing timing data...
api            INFO 	Using cached data for session_status_data
api            INFO 	Using cached data for track_status_data
api            INFO 	Using cached data for car_data
api            INFO 	Using cached data for position_data
api            INFO 	Using cached data for weather_data
api            INFO 	Using cached data for race_control_messages
core           INFO 	Finished loading data for 20 drivers: ['63', '44', '55', '16', '14', '1', '11', '31', '77', '18', '5', '24', '47', '10', '23', '6', '22', '4', '20', '3']
core           INFO 	Loading data for São Paulo Grand Prix - Race [v2.3.0]
api            INFO 	Using cached data for driver_info
api            INFO 	Using cached data for timing_data
api            INFO 	Using cached data for timing_app_data
core           INFO 	Processing timing data...
api            INFO

...

unused columns exists that are not in the dataframe


core        WARNING 	No lap data for driver 22
api            INFO 	Using cached data for car_data
api            INFO 	Using cached data for position_data
api            INFO 	Using cached data for weather_data
api            INFO 	Using cached data for race_control_messages
core           INFO 	Finished loading data for 19 drivers: ['1', '16', '55', '11', '63', '31', '4', '10', '20', '44', '24', '27', '18', '23', '77', '14', '3', '6', '22']
core           INFO 	Loading data for Abu Dhabi Grand Prix - Race [v2.3.0]
api            INFO 	Using cached data for driver_info
api            INFO 	Using cached data for timing_data
api            INFO 	Using cached data for timing_app_data
core           INFO 	Processing timing data...
api            INFO 	Using cached data for session_status_data
api            INFO 	Using cached data for track_status_data


unused columns exists that are not in the dataframe


api            INFO 	Using cached data for car_data
api            INFO 	Using cached data for position_data
api            INFO 	Using cached data for weather_data
api            INFO 	Using cached data for race_control_messages
core           INFO 	Finished loading data for 20 drivers: ['1', '16', '11', '55', '63', '4', '31', '18', '3', '5', '22', '24', '23', '10', '77', '47', '20', '44', '6', '14']


unused columns exists that are not in the dataframe
84.63694739341736 seconds


<h2>Caching of data and API calls

optional step that helps save time and computation when making the same API calls
<a href="https://theoehrly.github.io/Fast-F1/api.html?highlight=cache#fastf1.api.Cache">Caching Link</a>


<h1>Useful Links

<a href="https://theoehrly.github.io/Fast-F1/events.html#sessionidentifier">Session Identifiers</a>

Grand Prix can be refered to as the round number or the name with no spaces

In [ ]:
# fastf1.Cache.enable_cache("./cache")


In [ ]:
# session = fastf1.get_session(2022, 'Abu Dhabi', 'R')
# session.load()

# best_bottas = session.laps.pick_driver('HAM').pick_fastest()

In [ ]:
# ham = session.laps.pick_driver('HAM')
# ham_laps = session.laps.pick_driver('HAM').Compound
# compund_age = session.laps.pick_driver('HAM').TyreLife

In [ ]:
# test = session.laps

In [ ]:
# print(type(ham))

In [ ]:
driverAbbreviations = ["ALB","ALO","BOT","DEV","FIT","GAS","HAM","HUL","KUB","KVY","LAT","LEC","MAG","NOR","OCO","PER","RIC","RUS","SAI","SCH","STR","TSU","VER","VET","ZHO"]
pitstops = {}

# for driver in driverAbbreviations:
#     pitstops[driver] = pitstopLap((2022, 'Abu Dhabi', 'R'),driver)
#     print(pitstops)

<h1> Overtaking

 https://theoehrly.github.io/Fast-F1/core.html?highlight=overtake#fastf1.core.SessionResults

GridPosition - Position = ending

In [ ]:

#Create PySpark DataFrame from Pandas

sparkDF = spark.read.parquet("result.parquet.gzip")

sparkDF.count()
# sparkDF.where("Start < 10").show()
sparkDF.withColumn("Points", raceResultPoints.get("End"))
sparkDF.show()

In [ ]:
sparkDF.collect()
# key (Driver, Race, Year)